In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from gensim.corpora import WikiCorpus, MmCorpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument, Doc2VecVocab
from pprint import pprint
import multiprocessing
import sqlite3
from scipy.spatial import distance

In [ ]:
def preprocess(content):
    return content.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace("\'", " \' ")

documents = []
db = sqlite3.connect('../data/datasets/verbatim.sqlite')


with db:
    cr = db.cursor()
    cr.execute("select refid, content from verbatim where length(content) >= 200")
    
    for r in cr:
        refid, content = r
        documents.append(TaggedDocument(preprocess(content).split(), [refid]))

In [ ]:
cores = multiprocessing.cpu_count()
model = Doc2Vec(dm=0, dbow_words=1, vector_size=50, window=8, min_count=5, epochs=50, workers=cores)

In [ ]:
model.build_vocab(documents)
print(str(model))

In [ ]:
%time model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)
model.save("models/flair/doc2vec.model")

In [ ]:
t = "talend data"
v = model.infer_vector(t.split(), epochs=10)
res = model.docvecs.most_similar([v], topn=10)
verbs = []
with db:
    for ref, score in res:
        cr = db.cursor()
        cr.execute("select content from verbatim where refid = '{}'".format(ref))
        verbs.append(preprocess(cr.fetchone()[0]))

verbs

In [ ]:
t = "talend data"
v = model.infer_vector(t.split(), epochs=10)
print(v)